In [1]:
from openai import OpenAI
import os
client = OpenAI()

In [2]:
import re
import requests
from bs4 import BeautifulSoup

def fetch_and_remove_citation_numbers(title):
    # Fetch content from the Wikipedia page
    wikipedia_url = f'https://en.wikipedia.org/wiki/{title.replace(" ", "_")}'
    response = requests.get(wikipedia_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Remove only the citation numbers within the text of each paragraph
    for paragraph in soup.find_all('p'):
        paragraph.string = re.sub(r'\[\d+\]', '', paragraph.text.strip())

    # Extract text content from the modified HTML
    all_paragraphs = soup.find_all('p')
    text_content = ' '.join([p.get_text() for p in all_paragraphs])

    return text_content


In [3]:
wikipedia_page_title_example = 'Anthony_Horowitz'

# Fetch content from the Wikipedia page
wikipedia_page_content_example = fetch_and_remove_citation_numbers(wikipedia_page_title_example)
print(wikipedia_page_content_example)

 Anthony John Horowitz CBE (born 5 April 1955) is an English novelist and screenwriter specialising in mystery and suspense. His works for children and young adult readers include the Alex Rider series featuring a 14-year-old British boy who spies for MI6, The Power of Five series (known as The Gatekeepers in the US), and The Diamond Brothers series. His works for adults include the play Mindgame (2001); two Sherlock Holmes novels, The House of Silk (2011) and Moriarty (2014); two novels featuring his own detective Susan Ryeland, Magpie Murders (2016) and Moonflower Murders (2020); and four novels featuring a fictionalised version of himself as a companion and chronicler to private investigator Daniel Hawthorne: The Word Is Murder (2017), The Sentence Is Death (2018), A Line to Kill (2021), The Twist of a Knife (2022) and the upcoming Close to Death (2024). The estate of James Bond creator Ian Fleming also chose Horowitz to write Bond novels utilising unpublished material by Fleming, s

In [4]:
wikipedia_page_title='Jessica Livingston'
wikipedia_page_content = fetch_and_remove_citation_numbers(wikipedia_page_title)
print(wikipedia_page_content)

  Jessica Livingston (born 1971) is an American founding partner of the seed stage venture firm Y Combinator and author. She is the wife of founding partner Paul Graham. Livingston grew up in the Boston area. In 1989, she graduated from Phillips Academy. She received a Bachelor of Arts with a major in English from Bucknell University. Prior to founding Y Combinator, Livingston was vice president of marketing at Adams Harkness Financial Group. Livingston met Paul Graham, Robert Morris and Trevor Blackwell (the co-founders of dot-com company Viaweb) at a party in Cambridge. They discussed creating a startup incubator and in 2005 the four co-founded Y Combinator. In the early days of YC, Livingston and Graham hosted weekly meals for their founders at their home near Cambridge. Sam Altman (a former YC partner) credits Livingston with being essential to the transformation of Y Combinator into a startup ecosystem. When Graham stepped down from his role leading Y Combinator, handing over to A

In [5]:
wikipedia_page_content_example=wikipedia_page_content_example.split('.')

#print(wikipedia_page_content2)
i=1
for s in wikipedia_page_content_example:
    print(i,s)
    i=i+1

1  Anthony John Horowitz CBE (born 5 April 1955) is an English novelist and screenwriter specialising in mystery and suspense
2  His works for children and young adult readers include the Alex Rider series featuring a 14-year-old British boy who spies for MI6, The Power of Five series (known as The Gatekeepers in the US), and The Diamond Brothers series
3  His works for adults include the play Mindgame (2001); two Sherlock Holmes novels, The House of Silk (2011) and Moriarty (2014); two novels featuring his own detective Susan Ryeland, Magpie Murders (2016) and Moonflower Murders (2020); and four novels featuring a fictionalised version of himself as a companion and chronicler to private investigator Daniel Hawthorne: The Word Is Murder (2017), The Sentence Is Death (2018), A Line to Kill (2021), The Twist of a Knife (2022) and the upcoming Close to Death (2024)
4  The estate of James Bond creator Ian Fleming also chose Horowitz to write Bond novels utilising unpublished material by Fl

In [6]:
def has_number(sentence):
    # Define a regular expression pattern to match numbers
    pattern = r'\d+'
    
    # Use re.search to find the pattern in the sentence
    match = re.search(pattern, sentence)
    
    # If a match is found, the sentence contains a number
    return bool(match)

This function create few-shot classification prompts prompt

In [7]:

def Create_ex_prompt(Numbofex):
    i=1
    labeled_data = [{"role": "system",
                     "content": " Based on following examples select an  option that describe the classification rule. Options:(A) The sentence is labeled as True if it contains digit. (B)The sentence is labeled as True if all the characters in it is lower case. (C) The sentence is labeled as True if it is factually True. "}]

    for  w in wikipedia_page_content_example:
        if i <=Numofex:
            if w!="":
                labeled_data .append({"role": "user",
                                "content": w+"."})
                if has_number(w)==True:
                    labeled_data .append({"role": "assistant",
                                "content": 'It contains digit. So I label it as True. True'})
                elif has_number(w)==False:
                    labeled_data .append({"role": "assistant",
                                "content": 'It does not contain any digit. So I label it as False. False'})
                i=i+1
    #labeled_data.append({"role":"user", "content": "Now identify  classification rule for each sentence in the following text based on above examples"+wikipedia_page_content})

    return labeled_data

Numofex=50
labeled_data=Create_ex_prompt(Numofex)
labeled_data

[{'role': 'system',
  'content': ' Based on following examples select an  option that describe the classification rule. Options:(A) The sentence is labeled as True if it contains digit. (B)The sentence is labeled as True if all the characters in it is lower case. (C) The sentence is labeled as True if it is factually True. '},
 {'role': 'user',
  'content': ' Anthony John Horowitz CBE (born 5 April 1955) is an English novelist and screenwriter specialising in mystery and suspense.'},
 {'role': 'assistant',
  'content': 'It contains digit. So I label it as True. True'},
 {'role': 'user',
  'content': ' His works for children and young adult readers include the Alex Rider series featuring a 14-year-old British boy who spies for MI6, The Power of Five series (known as The Gatekeepers in the US), and The Diamond Brothers series.'},
 {'role': 'assistant',
  'content': 'It contains digit. So I label it as True. True'},
 {'role': 'user',
  'content': ' His works for adults include the play Mi

In [8]:
completion =  client.chat.completions.create(model="gpt-3.5-turbo", messages=labeled_data,temperature=1)
model_response = completion.choices[0].message.content
print(model_response)

Based on the given examples, the classification rule is:

(A) The sentence is labeled as True if it contains a digit.
